In [ ]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
c=config.Config()

# other imports
import pandas as pd
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

In [ ]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='electricity') # sensor objects

# Remove some sensors
exclude = [
            '565de0a7dc64d8370aa321491217b85f' # 3E
          ]
solar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
exclude += solar

for s in sensors:
    if s.key in exclude:
        sensors.remove(s)

hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
hp.find_device('FL03001251').sensors

In [ ]:
dt_now = pd.Timestamp('now', tz='UTC')
dt_start = dt_now - pd.Timedelta(hours=24)
df = hp.get_data(sensors=sensors, head=dt_start)

In [ ]:
df_total = pd.DataFrame(df.sum()/60/1000)
df_total.columns=['sum']
print df_total.describe()
print df_total


In [ ]:
df_total.sort(columns=['sum'], axis=0, inplace=True)

In [ ]:
df_total['percentile'] = np.array(range(len(df_total)))/float(len(df_total))

In [ ]:
np.array(range(len(df_total)))/float(len(df_total))

In [ ]:
df_total['flm'] = [hp.find_sensor(sensor).device.key for sensor in df_total.index]

In [ ]:
df_total

In [ ]:
threshhold = {"analyse1":
 {"default":0.8,
  "3aa40023a1683bfec0f01cce8d45fc70":0.5
 }
}

In [ ]:
import json
json.dump(threshhold, open("alerts.cfg", 'w'))

In [ ]:
def get_threshold(analyse, sensor_key):
    threshold = json.load("alerts.cfg")
    default = threshold[analyse]['default']